In [1]:
import numpy as np
import time

## Network architecture
NUM_INPUT = 784  # Number of input neurons
NUM_OUTPUT = 10  # Number of output neurons
NUM_CHECK = 5  # Number of examples on which to check the gradient

## Hyperparameters
NUM_HIDDEN = 50
LEARNING_RATE = 0.05
BATCH_SIZE = 64
NUM_EPOCH = 40

print("NUM_HIDDEN: ", NUM_HIDDEN)
print("LEARNING_RATE: ", LEARNING_RATE)
print("BATCH_SIZE: ", BATCH_SIZE)
print("NUM_EPOCH: ", NUM_EPOCH)

# Given a vector w containing all the weights and biased vectors, extract
# and return the individual weights and biases W1, b1, W2, b2.
def unpack (w):
    W1 = np.reshape(w[:NUM_INPUT * NUM_HIDDEN],(NUM_INPUT,NUM_HIDDEN))
    w = w[NUM_INPUT * NUM_HIDDEN:]
    b1 = np.reshape(w[:NUM_HIDDEN], NUM_HIDDEN)
    w = w[NUM_HIDDEN:]
    W2 = np.reshape(w[:NUM_HIDDEN*NUM_OUTPUT], (NUM_HIDDEN,NUM_OUTPUT))
    w = w[NUM_HIDDEN*NUM_OUTPUT:]
    b2 = np.reshape(w,NUM_OUTPUT)
    return W1, b1, W2, b2

# Given individual weights and biases W1, b1, W2, b2, concatenate them and
# return a vector w containing all of them.
def pack (W1, b1, W2, b2):
    W1_ = np.reshape(W1,NUM_INPUT*NUM_HIDDEN)
    # print(W1_.shape)
    W2_ = np.reshape(W2,NUM_HIDDEN*NUM_OUTPUT)
    # print(W2_.shape)
    w = np.concatenate((W1_,b1, W2_, b2))
    # print(w.shape)
    return w

# Load the images and labels from a specified dataset (train or test).
def loadData (which):
    images = np.load("./data/mnist_{}_images.npy".format(which))
    labels = np.load("./data/mnist_{}_labels.npy".format(which))
    return images, labels

## 1. Forward Propagation
# Given training images X, associated labels Y, and a vector of combined weights
# and bias terms w, compute and return the cross-entropy (CE) loss.

def fCE (X, Y, w):
    # print(X.shape)
    W1, b1, W2, b2 = unpack(w)
    
    z1 = np.dot(X, W1) + b1
    h1 = np.maximum(0, z1)
    z2 = np.dot(h1, W2) + b2
    exp_z2 = np.exp(z2)
    exp_sum = np.sum(exp_z2, axis=1).reshape(-1, 1)
    Y_pred = exp_z2 / exp_sum
    
    loss = -np.sum(np.log(Y_pred) * Y) / X.shape[0]
    return loss

## 2. Backward Propagation
# Given training images X, associated labels Y, and a vector of combined weights
# and bias terms w, compute and return the gradient of fCE. 
def gradCE (X, Y, w):
    W1, b1, W2, b2 = unpack(w)
    
    z1 = np.dot(X, W1) + b1
    h1 = np.maximum(0, z1)
    z2 = np.dot(h1, W2) + b2
    exp_z2 = np.exp(z2)
    exp_sum = np.sum(exp_z2, axis=1).reshape(-1, 1)
    Y_pred = exp_z2 / exp_sum
    
    delta_W_2 = np.dot(h1.T, Y_pred-Y) / X.shape[0]
    delta_b_2 = np.sum(Y_pred-Y, axis=0) / X.shape[0]
    delta_W_1 = X.T.dot((Y_pred-Y).dot(W2.T) * np.sign(z1)) / X.shape[0]
    delta_b_1 = np.sum((Y_pred-Y).dot(W2.T) * np.sign(z1), axis=0) / X.shape[0]
    
    delta = pack(delta_W_1, delta_b_1, delta_W_2, delta_b_2)
    return delta

def calAccuracy (X, Y, w):
    W1, b1, W2, b2 = unpack(w)
    
    z1 = np.dot(X, W1) + b1
    h1 = np.maximum(0, z1)
    z2 = np.dot(h1, W2) + b2
    exp_z2 = np.exp(z2)
    exp_sum = np.sum(exp_z2, axis=1).reshape(-1, 1)
    Y_pred = exp_z2 / exp_sum
    
    Y_pred_agm = np.argmax(Y_pred, axis=1)
    Y_agm = np.argmax(Y, axis=1)
    
    acc = 0
    for pred, label in zip(Y_pred_agm, Y_agm):
        if pred == label:
            acc += 1
    acc /= X.shape[0]
    return acc

## 3. Parameter Update
# Given training and testing datasets and an initial set of weights/biases,
# train the NN.
def train(trainX, trainY, testX, testY, w):
    print("----------------------------------------------------------------------------------")
    for epoch in range(NUM_EPOCH):
        loss = 0
        permutation = np.random.permutation(trainX.shape[0])
        trainX = trainX[permutation, :]
        trainY = trainY[permutation]
        for batch_begin in range(len(trainX))[::BATCH_SIZE]:
            batch_end = min(trainX.shape[0], batch_begin+BATCH_SIZE)
            X = trainX[batch_begin:batch_end,:]
            Y = trainY[batch_begin:batch_end]
            loss += fCE(X, Y, w) * (batch_end-batch_begin)
            w -= LEARNING_RATE * gradCE(X, Y, w)
        loss /= trainX.shape[0]
        train_acc = calAccuracy(trainX, trainY, w)
        test_acc = calAccuracy(testX, testY, w)
        print(f'Epoch:{epoch+1}    Loss:{loss}    Train Acc:{train_acc}    Test Acc:{test_acc}')

if __name__ == "__main__":
    # Load data
    start_time = time.time()
    trainX, trainY = loadData("train")
    testX, testY = loadData("test")

    print("len(trainX): ", len(trainX))
    print("len(testX): ", len(testX))

    # Initialize weights randomly
    W1 = 2*(np.random.random(size=(NUM_INPUT, NUM_HIDDEN))/NUM_INPUT**0.5) - 1./NUM_INPUT**0.5
    b1 = 0.01 * np.ones(NUM_HIDDEN)
    W2 = 2*(np.random.random(size=(NUM_HIDDEN, NUM_OUTPUT))/NUM_HIDDEN**0.5) - 1./NUM_HIDDEN**0.5
    b2 = 0.01 * np.ones(NUM_OUTPUT)

    w = pack(W1, b1, W2, b2)
    print("Shape of w:",w.shape)
    
    # Train the network and report the accuracy on the training and test set.
    train(trainX, trainY, testX, testY, w)

NUM_HIDDEN:  50
LEARNING_RATE:  0.05
BATCH_SIZE:  64
NUM_EPOCH:  40
len(trainX):  10000
len(testX):  5000
Shape of w: (39760,)
----------------------------------------------------------------------------------
Epoch:1    Loss:1.3960728279634358    Train Acc:0.83    Test Acc:0.8118
Epoch:2    Loss:0.5445833239454475    Train Acc:0.8711    Test Acc:0.8576
Epoch:3    Loss:0.40233907378475225    Train Acc:0.9005    Test Acc:0.889
Epoch:4    Loss:0.341530731752719    Train Acc:0.9069    Test Acc:0.8968
Epoch:5    Loss:0.3056690256855643    Train Acc:0.9186    Test Acc:0.9106
Epoch:6    Loss:0.28022023961633397    Train Acc:0.9221    Test Acc:0.9116
Epoch:7    Loss:0.2592123217197053    Train Acc:0.9252    Test Acc:0.9072
Epoch:8    Loss:0.2425623555156651    Train Acc:0.9356    Test Acc:0.9182
Epoch:9    Loss:0.22609267597849703    Train Acc:0.9403    Test Acc:0.9248
Epoch:10    Loss:0.2124629558803862    Train Acc:0.9404    Test Acc:0.9234
Epoch:11    Loss:0.20002843931643274    Train Acc: